In [1]:
#author : FELLOUSSI Idriss
# Production, Consumption , Emissions of gaz in France from 2007 to 2016  

import sqlite3

def read_file(fileName):
    lines = ''
    try :
        f = open(fileName, 'r', encoding='utf-8')
        lines = f.readlines()
        f.close()
    except FileNotFoundError :
        print("Wrong file or file path".open)
    return lines

def insert_data(table, lines, connection):
    cursor = connection.cursor()
    for line in lines :
        values = line.split(",")
        req = "INSERT INTO " + table + " VALUES (" + values[0] + ", "
        entite = values[1][:-6] + "\""
        dept = values[1][-4:][:-2]
        req += entite + ", " + dept + ", "
        for i in range(2, len(values)-1) :
            req += values[i] + ", "
        req += values[-1] + ");"
        cursor.execute(req)
    connection.commit()

def insert_data_2(table, lines, connection):
    cursor = connection.cursor()
    for line in lines :
        values = line.split(",")
        entite = values[1]
        dept = values[0][:2]
        annee = 2016
        for i in range(4, 12) :
            req = "INSERT INTO " + table + " VALUES (" + str(annee) + ", \"" + entite + "\", "+ dept + ", " + values[i] + ");"
            annee -= 1
            cursor.execute(req)
    connection.commit()

connection = sqlite3.connect('ProjectDB.db')
req01 = "DROP TABLE IF EXISTS consommation;"
req02 = "DROP TABLE IF EXISTS emissions;"
req03 = "DROP TABLE IF EXISTS production;"
req04 = "DROP TABLE IF EXISTS population;"
req1 = "CREATE TABLE emissions( Annee DATE , Entité TEXT, Dept INTEGER, Activité TEXT, Energie TEXT, Variable TEXT, Valeur DECIMAL(12,2), Unite TEXT);"
req2 = "CREATE TABLE consommation(Annee DATE , Entité TEXT, Dept INTEGER, Activité TEXT, Energie TEXT, Variable TEXT, Valeur DECIMAL(12,2), Unite TEXT);"
req3 = "CREATE TABLE production(Annee TEXT, Entité TEXT, Dept INTEGER, Type_production TEXT, Filière_production TEXT,Filière_détaillée TEXT, Valeur DECIMAL(12,2), Unite TEXT);"
req4 = "CREATE TABLE population(Annee DATE , Entité TEXT, Dept INTEGER, Valeur INTEGER);"
cursor = connection.cursor()
cursor.execute(req01)
cursor.execute(req02)
cursor.execute(req03)
cursor.execute(req04)
cursor.execute(req1)
cursor.execute(req2)
cursor.execute(req3)
cursor.execute(req4)

In [2]:
#Step 1 : import data

lines = read_file("atmosud_communes_emissions_ges_induites.csv")
lines = lines[1:]
insert_data("emissions", lines, connection)

lines = read_file("atmosud_communes_energie_consommation.csv")
lines = lines[1:]
insert_data("consommation", lines, connection)

lines = read_file("atmosud_communes_energie_production.csv")
lines = lines[1:]
insert_data("production", lines, connection)

lines = read_file("pop_historique.csv")
lines = lines[1:]
insert_data_2("population", lines, connection)

In [3]:
#Step 2 : Create VIEWS
req_v01 = "DROP VIEW IF EXISTS consommation_2;"
req_v02 = "DROP VIEW IF EXISTS production_2;"
req_v03 = "DROP VIEW IF EXISTS energy_production_consommation;"
req_v04 = "DROP VIEW IF EXISTS energy_emissions;"
req_v05 = "DROP VIEW IF EXISTS energy_dashboard;"
req_v1 = "CREATE VIEW consommation_2 AS SELECT Annee, Entité, Dept, Valeur*12.99 AS Valeur FROM consommation WHERE Unite = \"tep\";"
req_v2 = "CREATE VIEW production_2 AS SELECT Annee, Entité, Dept, sum(Valeur) as Valeur FROM production GROUP BY Annee, Entité;"
req_v3 = "CREATE VIEW energy_production_consommation AS SELECT production_2.Annee, consommation_2.Entité,consommation_2.Dept ,production_2.Valeur AS production,consommation_2.Valeur AS consommation,(production_2.Valeur - (consommation_2.Valeur)) AS difference_en_MWh FROM production_2 JOIN consommation_2 ON production_2.Entité  = consommation_2.Entité and production_2.Annee = consommation_2.Annee ; "
req_v4 = "CREATE VIEW energy_emissions AS SELECT emissions.Annee , emissions.Entité , emissions.Dept , emissions.Valeur, emissions.Valeur/population.Valeur AS emission_par_habitat FROM emissions JOIN population ON  emissions.Annee = population.Annee and emissions.Entité = population.Entité and  emissions.Dept = population.Dept"
req_v5 = "CREATE VIEW energy_dashboard AS SELECT energy_production_consommation.Annee, energy_production_consommation.Entité, energy_production_consommation.Dept, energy_production_consommation.production, energy_production_consommation.consommation, energy_production_consommation.difference_en_MWh, energy_emissions.Valeur AS emissions, energy_emissions.emission_par_habitat FROM energy_production_consommation JOIN energy_emissions ON energy_production_consommation.Entité = energy_emissions.Entité and energy_production_consommation.Annee = energy_emissions.Annee and energy_production_consommation.Dept = energy_emissions.Dept"
cursor.execute(req_v01)
cursor.execute(req_v02)
cursor.execute(req_v03)
cursor.execute(req_v04)
cursor.execute(req_v05)
cursor.execute(req_v1)
cursor.execute(req_v2)
cursor.execute(req_v3)
cursor.execute(req_v4)
cursor.execute(req_v5)

In [14]:
#Step 3 : DESCRIBE the utility 
#Refer to Readme 

In [4]:
#Step 4 : IHM 
from tkinter import * 
from tkinter import ttk
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

class Table:  
    def __init__(self,root, list): 
        for i in range(len(list)):
            for j in range(len(list[0])):
                self.e = Entry(root, width=20, fg='black',
                               font=('Arial',12))
                 
                self.e.grid(row=i, column=j, padx=10, pady=10)
                self.e.insert(END, list[i][j])

cursor.execute("SELECT distinct Dept FROM energy_production_consommation;")
dept_list = cursor.fetchall()

root = Tk()
root.title("Energy Dashbording Application for PACA region")
root.geometry("1600x900")

main_frame = Frame(root)
main_frame.pack(fill=BOTH, expand=1)

canvas = Canvas(main_frame)
canvas.pack(side=LEFT, fill=BOTH, expand=1)

scrollbar = Scrollbar(main_frame, orient=VERTICAL, command=canvas.yview)
scrollbar.pack(side=RIGHT, fill=Y)

canvas.configure(yscrollcommand=scrollbar.set)
canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

second_frame = Frame(canvas)

canvas.create_window((0,0), window=second_frame, anchor="nw")

#t = Table(second_frame, res[:50])

label_dep = Label(second_frame, text = "Veuillez choisir un département !")
label_dep.pack()
list_dep = ttk.Combobox(second_frame, values=dept_list)
list_dep.current(0)
list_dep.pack()

label_com = Label(second_frame, text = "Veuillez choisir une commune !")
label_com.pack()
list_com = ttk.Combobox(second_frame, values=[])
list_com.pack()

def action_dept(event):
    dept = list_dep.get()
    cursor.execute("SELECT distinct Entité FROM energy_production_consommation WHERE Dept = " + dept + ";")
    com_list = cursor.fetchall()
    list_com['values'] = com_list
    list_com.current(0)

f = Figure(figsize=(16,6), dpi=100)
a = f.add_subplot(131)
b = f.add_subplot(133)
c = f.add_subplot(132)
canvas_f = FigureCanvasTkAgg(f, second_frame)

def action_com(event):
    dept = list_dep.get()
    com = list_com.get()
    a.clear()
    b.clear()
    c.clear()
    cursor.execute("SELECT Annee, consommation FROM energy_production_consommation WHERE Dept = " + dept + " and Entité = \"" + com + "\";")
    conso = cursor.fetchall()
    x_conso = [conso[i][0] for i in range(len(conso))]
    y_conso = [conso[i][1] for i in range(len(conso))]
    cursor.execute("SELECT Annee, production FROM energy_production_consommation WHERE Dept = " + dept + " and Entité = \"" + com + "\";")
    prod = cursor.fetchall()
    x_prod = [prod[i][0] for i in range(len(prod))]
    y_prod = [prod[i][1] for i in range(len(prod))]
    cursor.execute("SELECT Annee, emission_par_habitat FROM energy_emissions WHERE Dept = " + dept + " and Entité = \"" + com + "\";")
    emis = cursor.fetchall()
    x_emis = [emis[i][0] for i in range(len(emis))]
    y_emis = [emis[i][1] for i in range(len(emis))]
    cursor.execute("SELECT Annee, difference_en_MWh FROM energy_production_consommation WHERE Dept = " + dept + " and Entité = \"" + com + "\";")
    diff = cursor.fetchall()
    x_diff = [diff[i][0] for i in range(len(diff))]
    y_diff = [diff[i][1] for i in range(len(diff))]

    a.plot(x_conso, y_conso, color='red', label='consommation en MWh')
    a.plot(x_prod, y_prod, color='green', label='production en MWh')
    a.legend(bbox_to_anchor=(1.05, 1),loc='upper right', borderaxespad=0.)
    a.set_xlabel("Année")
    a.set_ylabel("MWh")
    a.set_title("Consommation et production d'énergies \nà " + com + " ( " + dept + " ) en MWh par année", fontsize=8)

    b.plot(x_emis, y_emis)
    b.set_xlabel("Année")
    b.set_ylabel("kg")
    b.set_title("Emissions de CO2 par habitant \nà " + com + " ( " + dept + " ) en Kg par année", fontsize=8)
    
    c.bar(x_diff, y_diff)
    c.set_xlabel("Année")
    c.set_ylabel("MWh")
    c.set_title("Difference entre la production et la consommation d'énergie \nà " + com + " ( " + dept + " ) en Kg par année", fontsize=8)
    canvas_f.draw()
    canvas_f.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)

list_dep.bind("<<ComboboxSelected>>", action_dept)
list_com.bind("<<ComboboxSelected>>", action_com)

root.mainloop()